In [16]:
import matplotlib.pyplot as plt
from scipy.interpolate import InterpolatedUnivariateSpline
from scipy.signal import wiener
from scipy.ndimage import filters
import numpy as np

ImportError: cannot import name 'FitLogic' from 'tools' (d:\Pierre\Documents\Projet Cavity\Software\qudi\env\lib\site-packages\tools\__init__.py)

In [11]:
# Set this flag to True if you want to plot the results
plot_results = True
# This is the number of repetitions for each test function
repetitions = 100

In [12]:
def exponentialdecay_testing():
    #generation of data for testing
    x_axis = np.linspace(1, 51, 20)
    x_nice = np.linspace(x_axis[0], x_axis[-1], 100)
    
    mod, params = fitlogic.make_decayexponential_model()

    params['amplitude'].value = -100 + abs(np.random.normal(0,200))
    params['lifetime'].value = 1 + abs(np.random.normal(0,20))
    params['offset'].value = 1 + abs(np.random.normal(0, 200))

    data_noisy = (mod.eval(x=x_axis, params=params) + 7* np.random.normal(size=x_axis.shape))
    
    result = fitlogic.make_decayexponential_fit(
        x_axis=x_axis,
        data=data_noisy,
        estimator=fitlogic.estimate_decayexponential)
    
    data = data_noisy
    offset = data[-max(1,int(len(x_axis)/10)):].mean()

    #substraction of offset
    if data[0]<data[-1]:
        data_level = offset - data
    else:
        data_level = data - offset
    for i in range(0, len(x_axis)):
        if data_level[i] <= data_level.std():
            break
    
    try:
        data_level_log = np.log(data_level[0:i])
        linear_result = fitlogic.make_linear_fit(
            x_axis=x_axis[0:i],
            data=data_level_log,
            estimator=fitlogic.estimate_linear)
        if plot_results:
            plt.plot(x_axis[0:i], data_level_log, 'ob')
            plt.plot(x_axis[0:i], linear_result.best_fit,'-r')
            plt.plot(x_axis[0:i], linear_result.init_fit,'-y')
            plt.show()
    except:
        if plot_results:
            plt.plot(x_axis, np.log(data_level), 'or')
            plt.show()
        
    if plot_results:
        plt.plot(x_axis, data_noisy, 'ob')
        plt.plot(x_nice, mod.eval(x=x_nice, params=params), '-g')
        plt.plot(x_axis, result.init_fit, '-y', linewidth=2.0)
        plt.plot(x_axis, result.best_fit, '-r', linewidth=2.0)
        plt.show()

In [13]:
for i in range(repetitions):
    exponentialdecay_testing()

NameError: name 'fitlogic' is not defined

In [ ]:
def bareexponentialdecay_testing():
    #generation of data for testing
    x_axis = np.linspace(1, 100, 70)
    x_nice = np.linspace(x_axis[0], x_axis[-1], 200)

    mod, params = fitlogic.make_bareexponentialdecay_model()

    params['lifetime'].value = 1 + abs(np.random.normal(0,25))

    data_noisy = abs(mod.eval(x=x_axis, params=params) + 0.125 * np.random.normal(size=x_axis.shape))

    nice_data = mod.eval(x=x_nice, params=params)

    offset = data_noisy.min()

    leveled_data = data_noisy - offset

    if plot_results:
        plt.figure()
        plt.plot(x_nice, nice_data, label='ref exp. decay data no offest')
        plt.plot(x_nice, nice_data + 1, label='ref exp. decay data +1 offset')
        plt.legend(bbox_to_anchor=(0., 1.02, 1., .102), loc=3,
                   ncol=2, mode="expand", borderaxespad=0.,
                   prop={'size':12}, title='ref nice data')
        plt.show()

        plt.figure()
        plt.plot(x_nice, np.log(nice_data), label='ref exp. decay data no offest, log')
        plt.plot(x_nice, np.log(nice_data+1), label='ref exp. decay data +1 offset, log')
        plt.legend(bbox_to_anchor=(0., 1.02, 1., .102), loc=3,
                   ncol=2, mode="expand", borderaxespad=0.,
                   prop={'size':12}, title='ref nice data, log')
        plt.show()


    data_log = np.log(leveled_data)
    
    # Get all indices where entry is infinity  
    indices = np.where(np.isinf(data_log))

    # and remove those value, since they screw up the fit:
    x_axis = np.delete(x_axis, indices)
    data_noisy = np.delete(data_noisy, indices)
    data_log = np.delete(data_log, indices)

    linear_result = fitlogic.make_linear_fit(
        x_axis=x_axis,
        data=data_log,
        estimator=fitlogic.estimate_linear)
    
    if plot_results:
        plt.figure()
        plt.plot(x_axis, data_log, 'ob', label='logarithmic data')
        plt.plot(x_axis, linear_result.best_fit,'-r', label='best fit')
        plt.plot(x_axis, linear_result.init_fit,'-y', label='initial fit')
        plt.xlabel('Time x')
        plt.ylabel('signal')
        plt.legend(bbox_to_anchor=(0, 1.02, 1, .102), loc=3, ncol=2, mode="expand", borderaxespad=0)
        plt.show()

    result = fitlogic.make_decayexponential_fit(
        x_axis=x_axis,
        data=data_noisy,
        estimator=fitlogic.estimate_decayexponential)

    if plot_results:
        plt.figure()
        plt.plot(x_axis, data_noisy, 'ob',label='noisy data')
        plt.plot(x_nice, mod.eval(x=x_nice, params=params), '-g', label='simulated data')
        plt.plot(x_axis, result.init_fit, '-y', linewidth=1.0, label='initial values')
        plt.plot(x_axis, result.best_fit, '-r', linewidth=1.0, label='best fit')
        plt.legend(bbox_to_anchor=(0, 1.02, 1, .102), loc=3, ncol=2, mode="expand", borderaxespad=0)
        plt.xlabel('Time x')
        plt.ylabel('signal')
        plt.show()

In [ ]:
for i in range(repetitions):
    bareexponentialdecay_testing()

In [ ]:
def stretchedexponentialdecay_testing():
    x_axis = np.linspace(0, 51, 100)
    x_nice = np.linspace(x_axis[0], x_axis[-1], 100)

    mod, params = fitlogic.make_decayexponentialstretched_model()

    params['beta'].value = 2 + abs(np.random.normal(0,0.5))
    params['amplitude'].value = 10 #- abs(np.random.normal(0,20))
    params['lifetime'].value =1 + abs(np.random.normal(0,30))
    params['offset'].value = 1 + abs(np.random.normal(0, 20))

    data_noisy = (mod.eval(x=x_axis, params=params) + 1.5* np.random.normal(size=x_axis.shape))

    result = fitlogic.make_decayexponentialstretched_fit(
        x_axis=x_axis,
        data=data_noisy,
        estimator=fitlogic.estimate_decayexponentialstretched)

    data = data_noisy
    #calculation of offset
    offset = data[-max(1,int(len(x_axis)/10)):].mean()
    if data[0]<data[-1]:
        params['amplitude'].max = 0-data.std()
        data_sub = offset - data
    else:
        params['amplitude'].min = data.std()
        data_sub = data-offset

    amplitude = (data_sub.max()
                 - data_sub[-max(1, int(len(x_axis)/10)):].mean() 
                 - data_sub[-max(1,int(len(x_axis)/10)):].std())
    
    data_level = data_sub / amplitude

    a = 0
    b = len(data_sub)
    for i in range(0,len(data_sub)):
        if data_level[i]>=1:
            a=i+1
        if data_level[i] <=data_level.std():
            b=i
            break

    try:
        double_lg_data = np.log(-np.log(data_level[a:b]))

        #linear fit, see linearmethods.py
        X = np.log(x_axis[a:b])
        linear_result = fitlogic.make_linear_fit(axis=X, data=double_lg_data, add_parameters= None)
        
        if plot_results:
            plt.plot(np.log(x_axis),np.log(-np.log(data_level)),'ob')
            plt.plot(np.log(x_axis[a:b]),linear_result.best_fit,'-r')
            plt.plot(np.log(x_axis[a:b]),linear_result.init_fit,'-y')
            plt.show()
    except:
        print("except")
        
    if plot_results:
        plt.plot(x_axis, data_noisy, 'ob')
        plt.plot(x_nice, mod.eval(x=x_nice, params=params), '-g', label='original')
        plt.plot(x_axis, result.best_fit, '-r', linewidth=2.0, label='best fit')
        plt.plot(x_axis, result.init_fit, '-y', linewidth=2.0, label='init fit')
        plt.legend(bbox_to_anchor=(0, 1.02, 1, .102), loc=3, ncol=2, mode="expand", borderaxespad=0)
        plt.show()

In [ ]:
for i in range(repetitions):
    stretchedexponentialdecay_testing()